# UCS Data Pipeline: Standardization & Normalization

**Dataset:** Union of Concerned Scientists (UCS) Satellite Database  
**Objective:** Prepare active satellite registry data for merger with SATCAT.

### **The Engineering Challenge**
The UCS database is human-maintained, leading to significant inconsistencies in categorical fields. To make this data machine-readable for our "Kessler Syndrome" analysis, we must implement a strict cleaning pipeline:
1.  **Ingestion & Sanitization:** Load raw data and neutralize whitespace/character artifacts.
2.  **Normalization:** Standardize "Country of Operator" and "Users" to ensure categorical consistency.
3.  **Physical Validation:** Enforce orbital mechanics constraints (e.g., Apogee vs. Perigee).
4.  **Mass Imputation:** Address missing values using the "ISS Exception" and grouped median fills.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Markdown, display

### **Stage 1: Ingestion & String Sanitization**
**The Problem:** Raw human-maintained data often contains hidden whitespace and character artifacts (e.g., " USA " vs "USA"), which causes silent failures during categorical grouping.

**The Solution:** Use a **lambda-based stripping** operation to strictly trim whitespace from all text-based columns and headers, ensuring a clean baseline for the pipeline.

In [2]:
ucs_sats_messy = pd.read_csv('../data/original/UCS-Satellite-Database.csv')
text_cols = ucs_sats_messy.select_dtypes(['object']).columns

ucs_sats_messy[text_cols] = ucs_sats_messy[text_cols].apply(lambda x: x.str.strip())
ucs_sats_messy.columns = ucs_sats_messy.columns.str.strip()

### **Stage 1.1: Strategic Feature Selection**
**The Problem:** The raw UCS export contains numerous unpopulated placeholders (e.g., `Unnamed` columns) created by formatting artifacts in the original Excel file. These "Ghost Columns" inflate memory usage without adding information.

**The Solution:** Implement a **dynamic filter** to identify and drop all columns matching the `Unnamed` pattern, effectively sanitizing the dataframe structure.

In [3]:
unnamed_columns_dropped = [col for col in ucs_sats_messy.columns if 'Unnamed' in col]

if unnamed_columns_dropped:
    ucs_sats_messy.drop(columns=unnamed_columns_dropped, inplace=True)
    print(f"Dropped {len(unnamed_columns_dropped)} artifact columns (e.g., {unnamed_columns_dropped[0]}).")
else:
    print("No artifact columns found.")

Dropped 32 artifact columns (e.g., Unnamed: 28).


### **Stage 2: Enforcing Orbital Mechanics**
**The Problem:** Observational errors can result in physically impossible trajectories (Apogee < Perigee), and raw data often contains formatting artifacts (commas) that prevent numeric analysis.

**The Solution:**
* **Sanitize Numerics:** Remove string delimiters (commas) from `Perigee` and `Apogee`.
* **Physical Validation:** Implement a logical filter to ensure **Apogee (km) >= Perigee (km)**.

In [4]:
# Sanitize Numerics (Remove commas from strings)
ucs_sats_messy['Perigee (km)'] = ucs_sats_messy['Perigee (km)'].astype(str).str.replace(',', '', regex=False)
ucs_sats_messy['Apogee (km)'] = ucs_sats_messy['Apogee (km)'].astype(str).str.replace(',', '', regex=False)

# Convert to Float (Coerce errors to NaN)
ucs_sats_messy['Perigee (km)'] = pd.to_numeric(ucs_sats_messy['Perigee (km)'], errors='coerce')
ucs_sats_messy['Apogee (km)'] = pd.to_numeric(ucs_sats_messy['Apogee (km)'], errors='coerce')

# Drop missing values (We can't check physics if numbers are missing)
ucs_sats_messy.dropna(subset=['Perigee (km)', 'Apogee (km)'], inplace=True)

print("--- PRE-PATCH DIAGNOSTIC ---")
impossible_orbits = ucs_sats_messy[ucs_sats_messy['Apogee (km)'] < ucs_sats_messy['Perigee (km)']]
print(f"Satellites Violating Physics: {len(impossible_orbits)}")

if not impossible_orbits.empty:
    print("Violations Found:")
    print(impossible_orbits[['Name of Satellite, Alternate Names', 'Apogee (km)', 'Perigee (km)']].head(5))

# Fix known typo for Yaogan 35-5-1 (49.0 -> 499.0)
print("\n... Applying Manual Patch for Yaogan 35-5-1 ...\n")
ucs_sats_messy.loc[ucs_sats_messy['Name of Satellite, Alternate Names'] == 'Yaogan 35-5-1', 'Apogee (km)'] = 499.0

print("--- POST-PATCH DIAGNOSTIC ---")
impossible_orbits_after = ucs_sats_messy[ucs_sats_messy['Apogee (km)'] < ucs_sats_messy['Perigee (km)']]
print(f"Satellites Violating Physics: {len(impossible_orbits_after)}")

if impossible_orbits_after.empty:
    print("✅ SUCCESS: All physics violations resolved.")

# Only keeps valid rows. Since we fixed the error, we lose 0 satellites here.
ucs_sats_messy = ucs_sats_messy[ucs_sats_messy['Apogee (km)'] >= ucs_sats_messy['Perigee (km)']]

print(f"\nTotal Satellites Retained: {len(ucs_sats_messy)}")

--- PRE-PATCH DIAGNOSTIC ---
Satellites Violating Physics: 1
Violations Found:
     Name of Satellite, Alternate Names  Apogee (km)  Perigee (km)
7473                      Yaogan 35-5-1         49.0         493.0

... Applying Manual Patch for Yaogan 35-5-1 ...

--- POST-PATCH DIAGNOSTIC ---
Satellites Violating Physics: 0
✅ SUCCESS: All physics violations resolved.

Total Satellites Retained: 7553


### **Stage 3: Metadata Pruning (Schema Optimization)**
**The Problem:** Carrying extensive "Comments" and "Source" columns creates "Wide Data" that is inefficient for large-scale physics modeling and visualization.

**The Solution:**
* **Lean Schema:** We strictly drop unstructured metadata columns (`Source`, `Comments`) that are not required for kinetic analysis.
* **Safety Net:** We rely on the immutable `data/raw/` file as our permanent backup if we ever need to recover these columns.

In [5]:
# Define the list of metadata columns to remove
# These are non-analytical columns (URLs, text notes) that bloat the dataset.
metadata_cols = [
    'Source Used for Orbital Data', 'Source', 'Source.1', 'Source.2', 
    'Source.3', 'Source.4', 'Source.5', 'Source.6', 'Comments'
]

# Drop them directly from active memory
# We use errors='ignore' so this cell doesn't crash if we accidentally run it twice.
ucs_sats_messy.drop(columns=metadata_cols, inplace=True, errors='ignore')

print(f"✅ OPTIMIZATION COMPLETE: Dropped {len(metadata_cols)} metadata columns.")
print(f"Current Shape: {ucs_sats_messy.shape}")

✅ OPTIMIZATION COMPLETE: Dropped 9 metadata columns.
Current Shape: (7553, 27)


### **Stage 3.1: Categorical and Temporal Sanitization**

**The Problem:** Mixed-case strings, trailing whitespaces, and string-formatted dates create "duplicate" categories (e.g., "SpaceX" vs "SpaceX ") and prevent accurate time-series analysis.

**The Solution:**
* **Temporal Conversion:** Parse Date of Launch into standard datetime objects and remove the negligible number of rows missing this data to ensure a reliable timeline.
* **Case & Whitespace Normalization:** Force Class of Orbit to uppercase and strip() all major categorical columns to ensure unique, clean labels for grouping.
* **Logical Metadata Infill:** Perform early-stage cleanup on Contractor for vertically integrated companies and apply standard fallbacks for missing launch metadata to achieve 100% categorical density.

In [6]:
# Temporal Conversion & Age Calculation
ucs_sats_messy['Date of Launch'] = pd.to_datetime(ucs_sats_messy['Date of Launch'], errors='coerce')
ucs_sats_messy = ucs_sats_messy.dropna(subset=['Date of Launch'])

# Create Launch Year & Calculate Age
# We create Launch Year first, then use it for the age calculation
ucs_sats_messy['Launch Year'] = ucs_sats_messy['Date of Launch'].dt.year.astype(int)

current_year = 2026
ucs_sats_messy['Satellite Age (yrs)'] = current_year - ucs_sats_messy['Launch Year']

print(f"Temporal Check: Earliest launch in registry is {ucs_sats_messy['Launch Year'].min()}.")

# Case, Whitespace, and Country Normalization
# Expanded scrub list to include geopolitical metadata
scrub_cols = [
    'Operator/Owner', 'Contractor', 'Launch Site', 
    'Launch Vehicle', 'Purpose', 'Type of Orbit',
    'Country/Org of UN Registry', 'Country of Operator/Owner'
]

ucs_sats_messy['Class of Orbit'] = ucs_sats_messy['Class of Orbit'].str.upper()

for col in scrub_cols:
    if col in ucs_sats_messy.columns:
        # General whitespace cleanup
        ucs_sats_messy[col] = ucs_sats_messy[col].str.strip()
        
        # Geopolitical Standardization: Force Uppercase for grouping
        if 'Country' in col or 'Registry' in col:
            ucs_sats_messy[col] = ucs_sats_messy[col].str.upper()

# Fill SpaceX contractor gaps
ucs_sats_messy.loc[(ucs_sats_messy['Operator/Owner'] == 'SpaceX') & 
                   (ucs_sats_messy['Contractor'].isna()), 'Contractor'] = 'SpaceX'

# Geopolitical Proxy: If UN Registry is missing, fillna using 'Country of Operator/Owner'
ucs_sats_messy['Country/Org of UN Registry'] = ucs_sats_messy['Country/Org of UN Registry'].fillna(ucs_sats_messy['Country of Operator/Owner'])

# General fallbacks for remaining categorical gaps
ucs_sats_messy['Contractor'] = ucs_sats_messy['Contractor'].fillna('Unknown/Multiple')
ucs_sats_messy['Launch Site'] = ucs_sats_messy['Launch Site'].fillna('Unknown Site')
ucs_sats_messy['Launch Vehicle'] = ucs_sats_messy['Launch Vehicle'].fillna('Unknown Vehicle')
ucs_sats_messy['Country/Org of UN Registry'] = ucs_sats_messy['Country/Org of UN Registry'].fillna('UNKNOWN')

print(f"Sanitization Complete: {len(ucs_sats_messy)} satellites standardized.")
print(f"Ages calculated and geopolitical metadata normalized.")

Temporal Check: Earliest launch in registry is 1974.
Sanitization Complete: 7551 satellites standardized.
Ages calculated and geopolitical metadata normalized.


### **Stage 3.2: Universal Numeric Sanitization (The Physics 10)**

**The Problem:** High-fidelity physics modeling requires strict numeric types. However, fields like `Launch Mass`, `Period`, and `Perigee` often contain human-entered string artifacts—such as commas in "1,200" or non-numeric notes—that force Pandas to treat the entire column as an `object` (string).

**The Solution:**
* **Neutralize Delimiters:** Implement a universal string-replacement loop to strip commas across all 10 key physics columns.
* **Type Enforcement:** Utilize `pd.to_numeric` with `errors='coerce'`. This gracefully handles irregular entries (e.g., "15 years" or "~500") by converting them to `NaN`, ensuring the data is mathematically valid for Stage 4 calculations.
* **Immediate Diagnostic:** Execute a type-verification audit to confirm that every physics field has transitioned to `float64`.

**Impact:** This ensures that the "Physics Reconstruction Engine" in the next stage has a stable, purely numeric foundation to work from.

In [7]:
# The list of columns that must be numeric
columns_to_sanitize = [
    'Launch Mass (kg.)', 
    'Dry Mass (kg.)', 
    'Power (watts)',
    'Period (minutes)',
    'Expected Lifetime (yrs.)',
    'Perigee (km)',
    'Apogee (km)',
    'Eccentricity',
    'Inclination (degrees)',
    'Longitude of GEO (degrees)'
]

print(f"Sanitizing {len(columns_to_sanitize)} physics columns...")

# THE CLEANING LOOP
for col in columns_to_sanitize:
    if col in ucs_sats_messy.columns:
        # Force to string, remove commas, and coerce to float
        ucs_sats_messy[col] = ucs_sats_messy[col].astype(str).str.replace(',', '', regex=False)
        ucs_sats_messy[col] = pd.to_numeric(ucs_sats_messy[col], errors='coerce')

print("Numeric Sanitization Complete.")

print(f"\n{'Column (Original Name)':<30} | {'Current Type':<15} | {'Sample Value':<15} | {'Count'}")
print("-" * 75)

for col in columns_to_sanitize:
    if col in ucs_sats_messy.columns:
        dtype = str(ucs_sats_messy[col].dtype)
        sample = ucs_sats_messy[col].dropna().iloc[0] if not ucs_sats_messy[col].dropna().empty else "Empty"
        count = ucs_sats_messy[col].count()
        
        # We want to see 'float64' or 'int64'. 'object' is a failure.
        status_marker = "!!!" if 'object' in dtype else ""
        
        print(f"{col:<30} | {dtype:<15} | {sample:<15} | {count} {status_marker}")
    else:
        print(f"{col:<30} | NOT FOUND")

Sanitizing 10 physics columns...
Numeric Sanitization Complete.

Column (Original Name)         | Current Type    | Sample Value    | Count
---------------------------------------------------------------------------
Launch Mass (kg.)              | float64         | 22.0            | 7308 
Dry Mass (kg.)                 | float64         | 4.0             | 758 
Power (watts)                  | float64         | 4.5             | 557 
Period (minutes)               | float64         | 96.08           | 7498 
Expected Lifetime (yrs.)       | float64         | 0.5             | 5449 
Perigee (km)                   | float64         | 566.0           | 7551 
Apogee (km)                    | float64         | 576.0           | 7551 
Eccentricity                   | float64         | 0.00151         | 7547 
Inclination (degrees)          | float64         | 36.9            | 7550 
Longitude of GEO (degrees)     | float64         | 0.0             | 7549 


### **Stage 3.3: Primary Identifier Sanitization**

**The Problem:** Leading spaces in COSPAR Number and decimal artifacts in NORAD Number (e.g., "25544.0") will cause merge failures when connecting to the SATCAT.
**The Solution:**
* **String Scrubbing:** Strip whitespace and force COSPAR to uppercase.
* **Numeric Normalization:** Force NORAD Number through a numeric conversion to strip decimals before converting to a clean string. 
* **Null Recovery:** Re-convert "NAN" strings back to proper np.nan objects.

In [8]:
print("Sanitizing primary identifiers...")

# Clean COSPAR IDs (Removing leading spaces and normalizing case)
ucs_sats_messy['COSPAR Number'] = ucs_sats_messy['COSPAR Number'].astype(str).str.strip().str.upper()

# Clean NORAD IDs (Safety conversion to remove decimal artifacts)
# We coerce to numeric first to turn "25544.0" into 25544
ucs_sats_messy['NORAD Number'] = pd.to_numeric(ucs_sats_messy['NORAD Number'], errors='coerce')

# Drop rows where NORAD ID is missing (we cannot merge satellites without an ID)
ucs_sats_messy = ucs_sats_messy.dropna(subset=['NORAD Number'])

# Convert to clean integer strings (25544.0 -> 25544 -> "25544")
ucs_sats_messy['NORAD Number'] = ucs_sats_messy['NORAD Number'].astype(int).astype(str)

# Re-neutralize 'nan' strings for COSPAR
ucs_sats_messy['COSPAR Number'] = ucs_sats_messy['COSPAR Number'].replace('NAN', np.nan)

print(f"Identifier Sanitization Complete: {len(ucs_sats_messy)} records ready for merging.")

Sanitizing primary identifiers...
Identifier Sanitization Complete: 7551 records ready for merging.


### **Verification: Intermediate Pipeline Audit (The Quality Gate)**

**Objective:** This audit serves as a critical "Quality Gate" to verify the structural integrity of the dataframe before it enters the Stage 4 Physics Reconstruction Engine. 

**Validation Targets:**
1.  **Relational Integrity:** Confirm `Source` metadata was successfully decoupled and archived to `ucs_dropped.csv`.
2.  **Memory Optimization:** Ensure "Ghost Columns" and non-essential strings are purged.
3.  **Type Enforcement:** Confirm that mathematical fields (Perigee, Apogee, Mass) have been successfully sanitized of string artifacts and are ready for calculation.
4.  **Temporal Verification:** Validate the derivation of `Satellite Age (yrs)` to ensure the operational timeline is intact for future failure-rate modeling.

In [9]:
# Structural Check: Ghost & Metadata Columns
remaining_unnamed = [col for col in ucs_sats_messy.columns if 'Unnamed' in col]
metadata_to_check = [
    'Source Used for Orbital Data', 'Source', 'Source.1', 'Source.2', 
    'Source.3', 'Source.4', 'Source.5', 'Source.6', 'Comments'
]
remaining_metadata = [col for col in metadata_to_check if col in ucs_sats_messy.columns]

# Type Check: Comprehensive Physics Scan
physics_cols = ['Perigee (km)', 'Apogee (km)', 'Launch Mass (kg.)']
physics_status = all(pd.api.types.is_numeric_dtype(ucs_sats_messy[col]) for col in physics_cols)

# Temporal Check: Verify Date, Year, and Age
temporal_status = pd.api.types.is_datetime64_any_dtype(ucs_sats_messy['Date of Launch'])

# Verify Launch Year and age exist and are fully populated
year_age_status = all(col in ucs_sats_messy.columns and ucs_sats_messy[col].notna().all() 
                      for col in ['Launch Year', 'Satellite Age (yrs)'])

# Geopolitical Check: Verify Country Density
# Check if the registration proxy logic worked (should have 0 nulls now)
country_cols = ['Country/Org of UN Registry', 'Country of Operator/Owner']

# if any column (all(...) has nulls, geo_status will be false )
geo_status = all(ucs_sats_messy[col].notna().all() for col in country_cols)

# Relational Check: Identifier Whitespace
# check if the norad_id which is the primary key contains any spaces because it shouldnt
space_check = ucs_sats_messy['NORAD Number'].str.contains(' ').any()


# basic reporting output for visual verification/diagnostics
# its just a bunch of print statements with some emoji icons to make it less bland
print(f"{'--- INTERMEDIATE PIPELINE AUDIT ---':^45}")
print(f"{'CHECK':<26} | {'STATUS'}")
print("-" * 45)

# Report Temporal Integrity
print(f"{'Temporal Integrity':<26} | {'✅ VALID' if temporal_status and year_age_status else '❌ DATE/AGE ERROR'}")

# Report Geopolitical Density
print(f"{'Geopolitical Normalization':<26} | {'✅ DENSE' if geo_status else '❌ NULLS FOUND'}")

# Report Ghost Columns
print(f"{'Ghost Column Purge':<26} | {'✅ CLEAN' if not remaining_unnamed else f'❌ FOUND: {len(remaining_unnamed)}'}")

# Report Metadata Purge
print(f"{'Metadata Archive':<26} | {'✅ SUCCESS' if not remaining_metadata else '❌ FAILED'}")

# Report Physics Typing
print(f"{'Physics Type Enforcement':<26} | {'✅ NUMERIC' if physics_status else '❌ STRING ERROR'}")

# Report ID Sanitization
print(f"{'Identifier Sanitization':<26} | {'✅ WHITESPACE-FREE' if not space_check else '❌ SPACE DETECTED'}")

print("-" * 45)
print(f"Final Pre-Imputation Count: {len(ucs_sats_messy):,} satellites")

# Final logic gate
if not (physics_status and not space_check and year_age_status and geo_status):
    print("\n⚠️  WARNING: Quality Gate failed!")
else:
    print("\n🚀 PASS: Dataset is Physics-Ready.")

     --- INTERMEDIATE PIPELINE AUDIT ---     
CHECK                      | STATUS
---------------------------------------------
Temporal Integrity         | ✅ VALID
Geopolitical Normalization | ✅ DENSE
Ghost Column Purge         | ✅ CLEAN
Metadata Archive           | ✅ SUCCESS
Physics Type Enforcement   | ✅ NUMERIC
Identifier Sanitization    | ✅ WHITESPACE-FREE
---------------------------------------------
Final Pre-Imputation Count: 7,551 satellites

🚀 PASS: Dataset is Physics-Ready.


### **Stage 4.1: Addressing the Physics Transparency Gap (Mass & Power)**
**The Problem:** Critical physical properties (`Launch Mass`, `Dry Mass`, `Power`) are missing for significant portions of the registry. Deleting these rows would hide risk; leaving them empty breaks kinetic modeling.

**The Solution:**
* **The "White Whale" Exception:** Manually set the **ISS** mass (450,000 kg) to prevent it from skewing statistical medians.
* **Grouped Median Imputation:** Fill `Launch Mass` and `Power` using the median of satellites with similar **Orbit** and **Purpose**.
* **Physics-Informed Ratio:** Derive `Dry Mass` by calculating the typical *Dry-to-Wet Ratio* for each orbit class and applying it to the satellite's launch mass.

In [10]:
# THE ISS EXCEPTION (THE "WHITE WHALE")
# We manually set the station mass first because it is a unique outlier and would skew medians.
# We use '25544' as a string to match the sanitization and create a mask to make things easier.
iss_mask = ucs_sats_messy['NORAD Number'] == '25544'

# df.loc[row_mask, col_name] = value
ucs_sats_messy.loc[iss_mask, 'Launch Mass (kg.)'] = 450000
ucs_sats_messy.loc[iss_mask, 'Power (watts)'] = 84000
ucs_sats_messy.loc[iss_mask, 'Dry Mass (kg.)'] = 420000

print(f"ISS (NORAD 25544) manually patched: 450,000kg Mass | 420,000kg Dry | 84kW Power")

# IMPUTE LAUNCH MASS & POWER (GROUPED MEDIANS)
# Logic: Satellites with the same mission (Purpose) in the same region (Class of Orbit) 
# usually share similar chassis types (e.g., Starlink, OneWeb).
print("Imputing Launch Mass & Power via Grouped Medians...")
fill_cols = ['Launch Mass (kg.)', 'Power (watts)']

for col in fill_cols:
    # Calculate medians based on the specific peer group
    # in this case, the peer group is the combination of the class of orbit, and the purpose of the satellite
    medians = ucs_sats_messy.groupby(['Class of Orbit', 'Purpose'])[col].transform('median')
    ucs_sats_messy[col] = ucs_sats_messy[col].fillna(medians)

# Orbit Fallback for Mass
orbit_medians_mass = ucs_sats_messy.groupby('Class of Orbit')['Launch Mass (kg.)'].transform('median')
ucs_sats_messy['Launch Mass (kg.)'] = ucs_sats_messy['Launch Mass (kg.)'].fillna(orbit_medians_mass)

# Orbit Fallback for Power
orbit_medians_pwr = ucs_sats_messy.groupby('Class of Orbit')['Power (watts)'].transform('median')
ucs_sats_messy['Power (watts)'] = ucs_sats_messy['Power (watts)'].fillna(orbit_medians_pwr)

# Global Fallback for Mass (The Absolute Safety Net)
global_mass_median = ucs_sats_messy['Launch Mass (kg.)'].median()
ucs_sats_messy['Launch Mass (kg.)'] = ucs_sats_messy['Launch Mass (kg.)'].fillna(global_mass_median)

# Global Fallback for Power (The Absolute Safety Net)
global_power_median = ucs_sats_messy['Power (watts)'].median()
ucs_sats_messy['Power (watts)'] = ucs_sats_messy['Power (watts)'].fillna(global_power_median)

# IMPUTE DRY MASS (RATIO-DERIVED)
# We cannot use simple medians for Dry Mass because a 1kg CubeSat shouldn't 
# receive a 1,000kg median mass. We use the structural ratio instead.
print("Imputing Dry Mass via Orbit-Specific Mass Ratios...")

# Calculate existing ratios (Dry Mass / Launch Mass)
# Store the calculated ratio in a temp column we can drop later.
ucs_sats_messy['mass_ratio'] = ucs_sats_messy['Dry Mass (kg.)'] / ucs_sats_messy['Launch Mass (kg.)']

# Get the median ratio for each orbit (e.g., LEO sats vs. massive GEO commsats)
# using the new mass_ratio column we just calculated for every object in the dataframe.
ratio_medians = ucs_sats_messy.groupby('Class of Orbit')['mass_ratio'].transform('median')
ucs_sats_messy['mass_ratio'] = ucs_sats_messy['mass_ratio'].fillna(ratio_medians)

# Apply the ratio to the specific satellite's actual Launch Mass
estimated_dry_mass = ucs_sats_messy['Launch Mass (kg.)'] * ucs_sats_messy['mass_ratio']
ucs_sats_messy['Dry Mass (kg.)'] = ucs_sats_messy['Dry Mass (kg.)'].fillna(estimated_dry_mass)

# Drop the temporary ratio column now that we don't need it any more.
ucs_sats_messy.drop(columns=['mass_ratio'], inplace=True)

# FINAL PHYSICS AUDIT
print("\n--- Physics Gap Audit (Remaining Missing Values) ---")
print(f"Launch Mass: {ucs_sats_messy['Launch Mass (kg.)'].isnull().sum()}")
print(f"Dry Mass:    {ucs_sats_messy['Dry Mass (kg.)'].isnull().sum()}")
print(f"Power:       {ucs_sats_messy['Power (watts)'].isnull().sum()}")

ISS (NORAD 25544) manually patched: 450,000kg Mass | 420,000kg Dry | 84kW Power
Imputing Launch Mass & Power via Grouped Medians...
Imputing Dry Mass via Orbit-Specific Mass Ratios...

--- Physics Gap Audit (Remaining Missing Values) ---
Launch Mass: 0
Dry Mass:    0
Power:       0


### **Stage 4.2: Orbital & Lifecycle Sweep (The Final Gaps)**

**The Problem:** Secondary gaps in orbital elements (`Period`) and operational data (`Expected Lifetime`) prevent a total kinetic and temporal model.

**The Solution:**
1.  **Keplerian Derivation:** Use Kepler’s Third Law to mathematically calculate missing **Orbital Periods** from existing Perigee/Apogee data.
2.  **Lifecycle Imputation:** Fill missing **Expected Lifetimes** using medians grouped by `Class of Orbit`.
3.  **The "Dense" Registry:** Apply a final median sweep to ensure all 10 physics columns have 0 missing values.

In [11]:
#########################################################################################################
# AI Assisted Algorithm to Calculate Missing Periods                                                    #
#                                                                                                       #
# Kepler's Third Law (Calculating Period from Altitude)                                                 #
# Formula: T = 2 * pi * sqrt(a^3 / mu)  #                                                               #
# semi-major axis (a) = Earth_Radius + (Perigee + Apogee) / 2                                           #
#                                                                                                       #
# I don't understand enough about orbital mechanics to write this code myself,                          #          
# but I know that Kepler's Third Law relates the orbital period to the semi-major axis                  #
# and that the semi-major axis can be derived from the perigee and apogee altitudes.                    #
# We can use this knowledge to fill in missing orbital periods.                                         #
#                                                                                                       #               
# I wasn't sure how to represent the formula in a python function
# 2 * np.pi * np.sqrt(a**3 / mu) - wasnt sure how to do this part                                       #
# #######################################################################################################
earth_radius = 6378.137
mu = 398600.4418 # Earth's gravitational parameter (km^3/s^2)

def calculate_kepler_period(row):
    # Only calculate if Period is missing but we have altitudes
    # If either altitude is missing, we cannot compute the period.
    # if period_minutes is NaN and perigee and apogee are NOT nan, do stuff.
    if pd.isna(row['Period (minutes)']) and not pd.isna(row['Perigee (km)']) and not pd.isna(row['Apogee (km)']):
        
        # a = semi-major axis (Earth Radius + Average Altitude)
        a = earth_radius + ((row['Perigee (km)'] + row['Apogee (km)']) / 2)

        # T = 2 * pi * sqrt(a^3 / mu)
        period_seconds = 2 * np.pi * np.sqrt(a**3 / mu)

        return period_seconds / 60
    return row['Period (minutes)']

In [12]:
# use df.apply( func, axis ) to execute a function across every row
ucs_sats_messy['Period (minutes)'] = ucs_sats_messy.apply(calculate_kepler_period, axis=1)

# more basic reporting output for visual verification/diagnostics

print("Executing Final Physics Sweep...")

# These are all of the physics columns we need to double check and make sure are fully populated.
# Originally, during cleaning, we had to strip commas and coerce to numeric, which introduced NaNs.
# We also convert all 0s to NaNs because 0 is not a valid value for these physics parameters
# and the agg funcs we use later (median) would treat 0 as a valid number and skew results.
# agg funcs ignore NaNs but not 0s
sweep_cols = [
    'Expected Lifetime (yrs.)', 'Period (minutes)', 
    'Inclination (degrees)', 'Eccentricity', 
    'Perigee (km)', 'Apogee (km)', 'Longitude of GEO (degrees)'
]

# Source of truth logic order -> Original Dataframe, GroupBy( Orbit + Purpose ), GroupBy( Orbit ), Global Median
# first take values from the main dataframe, where available.
# then fill from peers (orbit class and purpose )

# loop the sweep_cols, if the column is in the main dataframe, do stuff
for col in sweep_cols:
    if col in ucs_sats_messy.columns:
        # if weve made it this far in the pipeline and its still null, fall back to orbit medians or global median.

        # Primary Fill: Grouped by Orbit Class
        orbit_medians = ucs_sats_messy.groupby('Class of Orbit')[col].transform('median')
        ucs_sats_messy[col] = ucs_sats_messy[col].fillna(orbit_medians)
        
        # Safety Fill: Global Median (In case Orbit Class was missing)
        ucs_sats_messy[col] = ucs_sats_messy[col].fillna(ucs_sats_messy[col].median())

print(f"\n{'Column':<30} | {'Status'}")
print("-" * 50)

all_physics = sweep_cols + [
    'Launch Mass (kg.)', 'Dry Mass (kg.)', 'Power (watts)', 
    'Satellite Age (yrs)', 'Launch Year'
]

for col in all_physics:
    if col in ucs_sats_messy.columns:
        missing = ucs_sats_messy[col].isnull().sum()
        print(f"{col:<30} | {'✅ COMPLETE' if missing == 0 else f'❌ {missing} MISSING'}")
    else:
        print(f"{col:<30} | ❌ NOT FOUND")

Executing Final Physics Sweep...

Column                         | Status
--------------------------------------------------
Expected Lifetime (yrs.)       | ✅ COMPLETE
Period (minutes)               | ✅ COMPLETE
Inclination (degrees)          | ✅ COMPLETE
Eccentricity                   | ✅ COMPLETE
Perigee (km)                   | ✅ COMPLETE
Apogee (km)                    | ✅ COMPLETE
Longitude of GEO (degrees)     | ✅ COMPLETE
Launch Mass (kg.)              | ✅ COMPLETE
Dry Mass (kg.)                 | ✅ COMPLETE
Power (watts)                  | ✅ COMPLETE
Satellite Age (yrs)            | ✅ COMPLETE
Launch Year                    | ✅ COMPLETE


### **Stage 5: Schema Alignment (Renaming & Type Finalization)**

**The Problem:** Raw UCS headers (e.g., `Name of Satellite, Alternate Names`) are too verbose for efficient coding and contain spaces/parentheses that can break certain SQL or Python operations.

**The Solution:** Implement a global **Renaming Schema** to transition the dataset into a strict **snake_case** format. 
1. **Primary Key Alignment:** Rename `NORAD Number` to `norad_id` to match the SATCAT pipeline.
2. **Physics Standardizing:** Shorten mass and power headers for programmatic speed.
3. **Identifier Cleaning:** Finalize `COSPAR Number` as `cospar_id`.

In [13]:
# Create rename mapping
column_mapping = {
    'Name of Satellite, Alternate Names': 'satellite_name',
    'Current Official Name of Satellite': 'official_name',
    'Country/Org of UN Registry': 'un_registry',
    'Country of Operator/Owner': 'country_operator',
    'Operator/Owner': 'owner',               # To be merged with SATCAT owner_code
    'Users': 'users',
    'Purpose': 'purpose',                    # Refined into primary_purpose
    'Class of Orbit': 'orbit_class',         # Standardized Regime: LEO, MEO, GEO
    'Type of Orbit': 'orbit_type',           # Standardized Geometry: Polar, Inclined
    'Longitude of GEO (degrees)': 'geo_longitude',
    'Perigee (km)': 'perigee_km',             # Standardized Physics (km)
    'Apogee (km)': 'apogee_km',               # Standardized Physics (km)
    'Eccentricity': 'eccentricity',           # Standardized Physics
    'Inclination (degrees)': 'inclination_degrees', # Standardized Physics (deg)
    'Period (minutes)': 'period_minutes',     # Standardized Physics (min)
    'Launch Mass (kg.)': 'launch_mass_kg',    # Core Kinetic Attribute
    'Date of Launch': 'launch_date',          # Standardized to datetime
    'Expected Lifetime (yrs.)': 'lifetime_years',
    'Contractor': 'contractor',
    'Country of Contractor': 'contractor_country',
    'Launch Site': 'launch_site',             # Matches SATCAT launch_site
    'Launch Vehicle': 'launch_vehicle',
    'COSPAR Number': 'cospar_id',             # Perfect Match with SATCAT cospar_id
    'NORAD Number': 'norad_id',               # Primary Merge Key (SATCAT norad_id)
    'Detailed Purpose': 'detailed_purpose',
    'Dry Mass (kg.)': 'dry_mass_kg',          # Derived Kinetic Attribute
    'Power (watts)': 'power_watts',
    'Satellite Age (yrs)': 'sat_age_years',   # Calculated for Simulation Year 2026
    'Launch Year': 'launch_year'              # Derived from launch_date
}

# Apply the Rename
ucs_sats_messy.rename(columns=column_mapping, inplace=True)

# Verification Check: Remaining messy headers
messy_headers = [col for col in ucs_sats_messy.columns if ' ' in col or '(' in col]

print(f"--- Schema Finalization Report ---")
print(f"Total Columns Standardized: {len(ucs_sats_messy.columns)}")
print(f"Messy Headers Remaining:    {'None (Full Clean)' if not messy_headers else messy_headers}")
print(f"Primary Merge Key:          { 'norad_id' in ucs_sats_messy.columns}")

--- Schema Finalization Report ---
Total Columns Standardized: 29
Messy Headers Remaining:    None (Full Clean)
Primary Merge Key:          True


### **Stage 6: Categorical Neutralization & Feature Engineering**

**The Problem:** Categorical data contains two distinct layers of "noise": 
1. **Density Gaps:** Sparse columns like `official_name` and `orbit_type` contain 600+ null values that can cause errors in string-processing functions.
2. **Complexity Gaps:** The `users` and `purpose` columns contain multi-stakeholder strings (e.g., "Government/Commercial/Military") that are difficult to query for statistical or kinetic analysis.

**The Solution:**
* **Density Sweep:** Implement a categorical "Neutralization" loop to fill all remaining text-based nulls with standardized fallbacks (e.g., "Unknown Sector", "Other/Misc").
* **Boolean Flags:** Decompose the `users` column into binary indicators (`is_commercial`, `is_government`, `is_military`, `is_civil`) to enable precise sector-based analysis.
* **Mission Standardization:** Map diverse mission descriptions into a controlled vocabulary (e.g., Mapping "Surveillance" and "Meteorological" to **"Earth Observation"**).

In [14]:
# Before engineering flags, we neutralize remaining nulls in text fields
# to ensure 100% density for the final kinetic model

# Categorical Fill Mapping, nulls/nans will be replaced with these values.
# We use 'Unknown' or similar neutral terms to avoid biasing analyses.
categorical_fills = {
    'official_name': ucs_sats_messy['satellite_name'], # Fallback to common name
    'users': 'Unknown Sector',
    'purpose': 'Unknown Purpose',
    'detailed_purpose': 'Not Specified',
    'contractor': 'Unknown/Multiple',
    'contractor_country': 'UNKNOWN',
    'cospar_id': 'NON-REGISTERED',
    'orbit_type': 'Other/Misc'  # neutralizes 651 nulls
}

for col, fill_value in categorical_fills.items(): # foreach (string col, string fill_value, in categorical_fills )
    if col in ucs_sats_messy.columns: # if the column exists in the dataframe, fill all nans with the fill_value
        ucs_sats_messy[col] = ucs_sats_messy[col].fillna(fill_value)

print(f"Categorical Sweep Complete. Remaining Nulls: {ucs_sats_messy.isnull().sum().sum()}")

# Create User Boolean Flags (The "Democratization" Columns)
# These flags allow queries like: "Show me Civil satellites with NO Government involvement"
# Basic boolean value determined by the presence of the flag keyword in the 'users' field.
ucs_sats_messy['is_commercial'] = ucs_sats_messy['users'].str.contains('Commercial', case=False, na=False).astype(int)
ucs_sats_messy['is_government'] = ucs_sats_messy['users'].str.contains('Government', case=False, na=False).astype(int)
ucs_sats_messy['is_military'] = ucs_sats_messy['users'].str.contains('Military', case=False, na=False).astype(int)
ucs_sats_messy['is_civil'] = ucs_sats_messy['users'].str.contains('Civil', case=False, na=False).astype(int)

# Standardize Primary Purpose (The "Mission")
def standardize_purpose(text):
    if pd.isna(text) or text == 'Unknown':
        return 'Unknown'
    
    # Take the first primary term if there are multiple (e.g. "Comms/Nav")
    primary = text.split('/')[0].strip()
    
    mapping = {
        'Earth Science': 'Earth Observation',
        'Meteorological': 'Earth Observation',
        'Surveillance': 'Earth Observation',
        'Earth': 'Earth Observation',
        'Earth/Space Observation': 'Earth Observation',
        'Space Observation': 'Space Science',
        'Technology Demonstration': 'Technology Development',
        'Mission Extension Technology': 'Technology Development',
        'Platform': 'Technology Development',
        'Satellite Positioning': 'Navigation',
        'Navigation': 'Navigation',
        'Communications': 'Communications',
        'Space Science': 'Space Science',
        'Educational': 'Educational'
    }
    return mapping.get(primary, primary)

ucs_sats_messy['primary_purpose'] = ucs_sats_messy['purpose'].apply(standardize_purpose)

# Logical Reordering (Move primary_purpose next to purpose for easy checking)
cols = list(ucs_sats_messy.columns)
cols.remove('primary_purpose')
target_index = cols.index('purpose')
cols.insert(target_index + 1, 'primary_purpose')
ucs_sats_messy = ucs_sats_messy[cols]

columns_to_show = ['satellite_name', 'purpose', 'primary_purpose']
diff_view = ucs_sats_messy[ucs_sats_messy['purpose'] != ucs_sats_messy['primary_purpose']][columns_to_show]

if not diff_view.empty:
    print(f"Standardized {len(diff_view)} complex mission labels into controlled vocabulary.")
    display(diff_view.head(10))
else:
    print("No complex labels found.")

Categorical Sweep Complete. Remaining Nulls: 0
Standardized 324 complex mission labels into controlled vocabulary.


,satellite_name,purpose,primary_purpose
12,ADLER-2,Earth Science,Earth Observation
51,ALE-2 (Astro Live Experiences-2),Earth Science,Earth Observation
75,ANDESITE Mule (Ad-Hoc Network Demonstration fo...,Space Science/Technology Demonstration,Space Science
83,AprizeSat 1 (LatinSat-C),Communications/Technology Development,Communications
84,AprizeSat 10 (exactView-13),Communications/Maritime Tracking,Communications
85,AprizeSat 2 (LatinSat-D),Communications/Technology Development,Communications
86,AprizeSat 3,Communications/Maritime Tracking,Communications
87,AprizeSat 4,Communications/Maritime Tracking,Communications
90,AprizeSat 8 (exactView-12),Communications/Maritime Tracking,Communications
91,AprizeSat 9 (exactView-11),Communications/Maritime Tracking,Communications


### **Stage 6.1: Orbit Class Standardization**
**The Problem:** The `orbit_class` column contains synonymous but inconsistent labels (e.g., "Low Earth Orbit" vs "LEO"). 

**The Solution:** Implement a mapping dictionary to consolidate all orbital regimes into four standardized categories: **LEO, MEO, GEO,** and **Elliptical**. This ensures compatibility with the SATCAT classification logic used in the next phase of the pipeline.

In [15]:
# Standardize Orbit Class (The Region)
ucs_sats_messy['orbit_class'] = ucs_sats_messy['orbit_class'].str.upper()

# create a mapping dictionary for the orbit classes we want to standardize just like we did with purposes
orbit_class_map = {
    'LEO': 'LEO',
    'GEO': 'GEO',
    'MEO': 'MEO',
    'ELLIPTICAL': 'Elliptical',
    'Elliptical': 'Elliptical'
}

# Standardize Orbit Type (The Geometry)
# and again we create another mapping dictionary but for the orbit types we want to standardize
orbit_type_map = {
    'Non-Polar Inclined': 'Inclined',
    'Sun-Synchronous': 'Polar',
    'Polar': 'Polar',
    'Equatorial': 'Equatorial',
    'Molniya': 'Eccentric',
    'Deep Highly Eccentric': 'Eccentric',
    'Elliptical': 'Eccentric',
    'Sun-Synchronous near polar': 'Polar',
    'Cislunar': 'Eccentric',
    'Retrograde': 'Inclined'
}

# apply the mappings to their appropriate columns and strip white space/fillna with neutral terms where needed
ucs_sats_messy['orbit_class'] = ucs_sats_messy['orbit_class'].str.strip().map(orbit_class_map).fillna('Unknown')
ucs_sats_messy['orbit_type'] = ucs_sats_messy['orbit_type'].str.strip().map(orbit_type_map).fillna('Other/Misc')

# print the value counts for each unique orbit class
print("\n--- Orbit Class Distribution ---")
print(ucs_sats_messy['orbit_class'].value_counts())

# print the total number of null values in the orbit_type column
# this serves as a visual check to ensure standardization and filling are complete
print("\n--- Orbit Type Density Check ---")
print(f"Nulls in orbit_type: {ucs_sats_messy['orbit_type'].isnull().sum()}")


--- Orbit Class Distribution ---
orbit_class
LEO           6759
GEO            590
MEO            143
Elliptical      59
Name: count, dtype: int64

--- Orbit Type Density Check ---
Nulls in orbit_type: 0


### **Stage 6.2: Final Integrity Polish & Constraint Enforcement**

**The Problem:** 1. **Relational Duplication:** The UCS registry occasionally contains duplicate entries for a single `norad_id` due to multi-name listings. Merging these would cause "Cartesian inflation" (exponentially increasing row counts) in the next notebook.
2. **Physical Outliers:** Imputation logic or raw data entry errors can occasionally produce non-physical values, such as a mass of `0` or negative `eccentricity`, which would crash kinetic energy calculations ($E_k = \frac{1}{2}mv^2$).

**The Solution:**
* **Deduplication:** Enforce a strict 1:1 relationship between `norad_id` and satellite records by pruning duplicates, ensuring a clean primary key for the SATCAT merger.
* **Physics Enforcement:** A final "Sanity Sweep" to force mass to the global median and eccentricity to a minimum of `0.0` (Circular), ensuring every asset adheres to the laws of orbital mechanics.

In [16]:
print("Executing Deep Scrub & Integrity Polish...")

# Global String Strip (The "Invisible Bug" Fix)
# Removes trailing spaces, tabs, and non-printing characters that break merges
for col in ucs_sats_messy.select_dtypes(include=['object']).columns:
    ucs_sats_messy[col] = ucs_sats_messy[col].astype(str).str.strip()
ucs_sats_messy = ucs_sats_messy.replace(r'[^\x00-\x7F]+', '', regex=True) # Zero-width space removal

# Temporal Sync (Eliminating "Age Drift")
# Ensures year and age are perfectly aligned to the launch date
ucs_sats_messy['launch_year'] = pd.to_datetime(ucs_sats_messy['launch_date']).dt.year
ucs_sats_messy['sat_age_years'] = (2026 - ucs_sats_messy['launch_year']).astype(int)

# Deduplication (Primary Key Integrity)
# Prevents row inflation during the SATCAT
initial_count = len(ucs_sats_messy)
ucs_sats_messy = ucs_sats_messy.drop_duplicates(subset=['norad_id'], keep='first')
dropped_dupes = initial_count - len(ucs_sats_messy)

# Physics Enforcement (The "Laws of Nature" Check)
# Fixes mass errors and impossible inclination/eccentricity values
mass_err = ucs_sats_messy['launch_mass_kg'] <= 0
ecc_err = ucs_sats_messy['eccentricity'] < 0

# Logic Check: If 'Polar' type but inclination is < 70, force to 90
polar_logic_err = (ucs_sats_messy['orbit_type'] == 'Polar') & (ucs_sats_messy['inclination_degrees'] < 70)

if mass_err.any():
    ucs_sats_messy.loc[mass_err, 'launch_mass_kg'] = ucs_sats_messy['launch_mass_kg'].median()
if ecc_err.any():
    ucs_sats_messy.loc[ecc_err, 'eccentricity'] = 0.0
if polar_logic_err.any():
    ucs_sats_messy.loc[polar_logic_err, 'inclination_degrees'] = 90.0

print(f"--- Results ---")
print(f"Deduplication: Dropped {dropped_dupes} duplicate records.")
print(f"Temporal Sync: Aligned {len(ucs_sats_messy)} records to Simulation Year 2026.")
print(f"Physics Check: Mass, Eccentricity, and Polar logic enforced.")
print(f"Final Record Count: {len(ucs_sats_messy)}")

Executing Deep Scrub & Integrity Polish...
--- Results ---
Deduplication: Dropped 9 duplicate records.
Temporal Sync: Aligned 7542 records to Simulation Year 2026.
Physics Check: Mass, Eccentricity, and Polar logic enforced.
Final Record Count: 7542


### **Stage 7: Pipeline Serialization & Executive Summary**

**Objective:** Finalize the active population for export and provide a technical audit of the registry's health.

We have successfully addressed four critical data gaps to create a simulation-ready dataset:
1. **The "Mass Transparency Gap":** Addressed via the **ISS Exception**, **Grouped Median Imputation**, and **Physics-Informed Ratios**, creating a high-fidelity mass reference for collision models.
2. **The "Metadata Consistency Gap":** Addressed via **Boolean Sector Flags** and **Mission Standardization**, transforming raw text into machine-readable categories.
3. **The "Orbital Regime Gap":** Standardized **`orbit_class`** and **`orbit_type`** to ensure 100% density for congestion modeling and regime-based filtering.
4. **The "Integrity Gap":** Deduplicated primary keys and enforced physical constraints (Mass > 0, Eccentricity $\ge$ 0) to ensure the registry acts as a stable, unique kinetic reference.

**Outcome:** This dataset is now normalized, validated, and exported as `ucs_cleaned.csv` for use in the Kessler Syndrome simulation.

In [ ]:
output_path = '../data/clean/ucs_cleaned.csv'

# research helped me learn how to format the markdown strings for specific formatting 
# when using f-strings in python I can
# use triple quotes to create multi-line strings and include markdown syntax directly within the string
# then I can use the f-string placeholders to insert calculated values into the markdown table
# finally I can print the entire markdown report as a single string and display it using display(Markdown(report))

total_rows = len(ucs_sats_messy)
comm_count = ucs_sats_messy['is_commercial'].sum()
mil_count  = ucs_sats_messy['is_military'].sum()
gov_count  = ucs_sats_messy['is_government'].sum()
civ_count  = ucs_sats_messy['is_civil'].sum()

# Temporal Metrics & Fleet Health
avg_age = ucs_sats_messy['sat_age_years'].mean()
zombie_count = ucs_sats_messy[ucs_sats_messy['sat_age_years'] > ucs_sats_messy['lifetime_years']].shape[0]
zombie_percent = (zombie_count / total_rows) if total_rows > 0 else 0

# Congestion Metrics (The Polar Alert)
# we dont have to use a mask here, we only end up using the 2 masks we created one time
# and then we only reference the polar_share varible one time
# but using a mask makes the code more readable and easier to understand
leo_mask = ucs_sats_messy['orbit_class'] == 'LEO'
polar_mask = ucs_sats_messy['orbit_type'] == 'Polar'
polar_leo_count = len(ucs_sats_messy[leo_mask & polar_mask])
polar_share = (polar_leo_count / len(ucs_sats_messy[leo_mask])) if total_rows > 0 else 0

# Calculate Mission Metrics
# spread the primary_purpose unique values down across the rows
# and then count the occurrences of each unique value
top_missions = ucs_sats_messy['primary_purpose'].value_counts().head(3)

# use .index and .value to access the 3 top missions names and associated counts
mission_one_name, mission_one_count = top_missions.index[0], top_missions.values[0]
mission_two_name, mission_two_count = top_missions.index[1], top_missions.values[1]
mission_three_name, mission_three_count = top_missions.index[2], top_missions.values[2]

# Calculate Timeline Range
earliest_launch = ucs_sats_messy['launch_date'].min().year # find the minimum (first) launch date year
latest_launch   = ucs_sats_messy['launch_date'].max().year # find the most recent (last) launch date year

physics_features = [
    'norad_id', 'cospar_id',                           # Primary Identifiers
    'launch_mass_kg', 'dry_mass_kg', 'power_watts',    # Physical Specs
    'period_minutes', 'perigee_km', 'apogee_km',       # Orbital Mechanics
    'inclination_degrees', 'eccentricity', 'geo_longitude',
    'orbit_class', 'orbit_type',                       # Orbital Regimes
    'launch_year', 'sat_age_years', 'lifetime_years',  # Temporal Metrics
    'un_registry', 'country_operator',                 # Geopolitical Metadata
    'primary_purpose'                                  # Engineered Mission Data
]

# syntax cheat sheet
# :---   : left align
# :---:  : center align
# ---:   : right align
# :,     : thousands separator
# :.1%   : percentage with 1 decimal place
# :.1f   : float with 1 decimal place
# :d     : integer
# :,d    : integer with thousands separator
# :,f    : float with thousands separator

# while i really hate languages that are not strongly typed
# i will say that python f-strings are pretty powerful and flexible for generating dynamic reports

report = f"""
### **UCS Pipeline Completion Report**
**Total Active Registry:** {total_rows:,} Satellites

#### **Fleet Timeline Summary**
| Metric | Value |
| :--- | :--- |
| **Oldest Active Asset** | {earliest_launch} |
| **Newest Active Asset** | {latest_launch} |
| **Active Span** | {latest_launch - earliest_launch} Years |

#### **Fleet Composition & Health**
| Metric | Value | Note |
| :--- | :--- | :--- |
| **Average Fleet Age** | {avg_age:.1f} Years | Simulation Year: 2026 |
| **End-of-Life Alert** | **{zombie_count:,} ({zombie_percent:.1%})** | Satellites exceeding design life ⚠️ |
| **Polar Congestion** | {polar_share:.1%} | Share of LEO in Polar Orbits |

#### **Sector Composition**
| Sector | Count | Share |
| :--- | :--- | :--- |
| **Commercial** | {comm_count:,} | {comm_count/total_rows:.1%} |
| **Military** | {mil_count:,} | {mil_count/total_rows:.1%} |
| **Government** | {gov_count:,} | {gov_count/total_rows:.1%} |
| **Civil** | {civ_count:,} | {civ_count/total_rows:.1%} |

#### **Primary Mission Breakdown**
| Top Mission | Count | Share |
| :--- | :--- | :--- |
| **1. {mission_one_name}** | {mission_one_count:,} | {mission_one_count/total_rows:.1%} |
| **2. {mission_two_name}** | {mission_two_count:,} | {mission_two_count/total_rows:.1%} |
| **3. {mission_three_name}** | {mission_three_count:,} | {mission_three_count/total_rows:.1%} |

#### **Data Quality & Density Engineering**
| Feature | Completeness | Method | Status |
| :--- | :--- | :--- | :--- |
"""

for feature in physics_features:
    coverage = ucs_sats_messy[feature].notna().mean()

    if feature in ['norad_id', 'cospar_id']:
        method = "Sanitized/Verified" 
    elif feature in ['un_registry', 'country_operator']:
        method = "Normalized/Proxy" 
    elif feature in ['sat_age_years', 'period_minutes', 'dry_mass_kg', 'launch_year']:
        method = "Derived/Calculated" 
    elif feature in ['orbit_class', 'orbit_type', 'primary_purpose']:
        method = "Standardized/Mapped" 
    else:
        method = "Grouped Median" 
        
    report += f"| **{feature.replace('_', ' ').title()}** | **{coverage:.1%}** | {method} | ✅ SUCCESS |\n"

display(Markdown(report))
ucs_sats_messy.to_csv(output_path, index=False)


### **UCS Pipeline Completion Report**
**Total Active Registry:** 7,542 Satellites

#### **Fleet Timeline Summary**
| Metric | Value |
| :--- | :--- |
| **Oldest Active Asset** | 1974 |
| **Newest Active Asset** | 2023 |
| **Active Span** | 49 Years |

#### **Fleet Composition & Health**
| Metric | Value | Note |
| :--- | :--- | :--- |
| **Average Fleet Age** | 6.5 Years | Simulation Year: 2026 |
| **End-of-Life Alert** | **3,834 (50.8%)** | Satellites exceeding design life ⚠️ |
| **Polar Congestion** | 41.2% | Share of LEO in Polar Orbits |

#### **Sector Composition**
| Sector | Count | Share |
| :--- | :--- | :--- |
| **Commercial** | 6,251 | 82.9% |
| **Military** | 613 | 8.1% |
| **Government** | 762 | 10.1% |
| **Civil** | 219 | 2.9% |

#### **Primary Mission Breakdown**
| Top Mission | Count | Share |
| :--- | :--- | :--- |
| **1. Communications** | 5,514 | 73.1% |
| **2. Earth Observation** | 1,310 | 17.4% |
| **3. Technology Development** | 441 | 5.8% |

#### **Data Quality & Density Engineering**
| Feature | Completeness | Method | Status |
| :--- | :--- | :--- | :--- |
| **Norad Id** | **100.0%** | Sanitized/Verified | ✅ SUCCESS |
| **Cospar Id** | **100.0%** | Sanitized/Verified | ✅ SUCCESS |
| **Launch Mass Kg** | **100.0%** | Grouped Median | ✅ SUCCESS |
| **Dry Mass Kg** | **100.0%** | Derived/Calculated | ✅ SUCCESS |
| **Power Watts** | **100.0%** | Grouped Median | ✅ SUCCESS |
| **Period Minutes** | **100.0%** | Derived/Calculated | ✅ SUCCESS |
| **Perigee Km** | **100.0%** | Grouped Median | ✅ SUCCESS |
| **Apogee Km** | **100.0%** | Grouped Median | ✅ SUCCESS |
| **Inclination Degrees** | **100.0%** | Grouped Median | ✅ SUCCESS |
| **Eccentricity** | **100.0%** | Grouped Median | ✅ SUCCESS |
| **Geo Longitude** | **100.0%** | Grouped Median | ✅ SUCCESS |
| **Orbit Class** | **100.0%** | Standardized/Mapped | ✅ SUCCESS |
| **Orbit Type** | **100.0%** | Standardized/Mapped | ✅ SUCCESS |
| **Launch Year** | **100.0%** | Derived/Calculated | ✅ SUCCESS |
| **Sat Age Years** | **100.0%** | Derived/Calculated | ✅ SUCCESS |
| **Lifetime Years** | **100.0%** | Grouped Median | ✅ SUCCESS |
| **Un Registry** | **100.0%** | Normalized/Proxy | ✅ SUCCESS |
| **Country Operator** | **100.0%** | Normalized/Proxy | ✅ SUCCESS |
| **Primary Purpose** | **100.0%** | Standardized/Mapped | ✅ SUCCESS |


## **Cleaned UCS Registry: Data Dictionary**

#### **1. Physical & Kinetic Properties (Imputed/Verified)**
These columns are the "Engine" of the model. All missing values have been filled using physics-informed logic and **enforced physical constraints** (Mass > 0, Eccentricity $\ge$ 0).

| Feature Name | Type | Description |
| :--- | :--- | :--- |
| `launch_mass_kg` | `float` | Total mass at launch. Filled via grouped medians. **Validated > 0.** |
| `dry_mass_kg` |    `float` | Mass without fuel. Derived using orbit-specific *Dry-to-Wet* ratios. |
| `power_watts` |    `float` | Electrical output. Standardized and imputed via grouped medians. |
| `lifetime_years` | `float` | Expected operational span. Imputed via orbit-class medians. |
| `sat_age_years` |  `int`   | Calculated age ($2026 - launch\_year$) for degradation modeling. |

#### **2. Orbital Mechanics (Keplerian Verified)**
The geometry of the satellite's path. These values are physically consistent and standardized for regime analysis.

| Feature Name | Type | Description |
| :--- | :--- | :--- |
| `period_minutes` | `float` | Time for one orbit. **Derived via Kepler's 3rd Law** ($T = 2\pi\sqrt{a^3/\mu}$) where missing. |
| `perigee_km` |      `float` | Closest point to Earth. Sanitized and imputed. |
| `apogee_km` |       `float` | Farthest point from Earth. Sanitized and imputed. |
| `inclination_degrees`| `float` | Angle relative to the equator. Essential for "Polar Congestion" analysis. |
| `eccentricity` |    `float` | Deviation from a perfect circle. **Validated $\ge$ 0.** |
| `orbit_class` |      `str`   | Standardized regime: **LEO, MEO, GEO, Elliptical**. |
| `orbit_type` |       `str`   | Standardized geometry: **Polar, Equatorial, Inclined, Eccentric**. |
| `geo_longitude` |  `float` | Fixed position for GEO sats; set to 0.0 for non-GEO payloads. |

#### **3. Categorical & Temporal Metadata**
Standardized labels used for filtering, merging, and geopolitical analysis.

| Feature Name | Type | Description |
| :--- | :--- | :--- |
| `norad_id` |         `str`      | **Primary Merge Key.** Cleaned and **Deduplicated** for 1:1 SATCAT join. |
| `cospar_id` |        `str`      | International designator. Sanitized and whitespace-stripped. |
| `launch_date` |      `datetime` | Precise date of launch. Standardized for time-series analysis. |
| `launch_year` |      `int`      | **Derived Year of Launch.** Used for SATCAT alignment. |
| `primary_purpose` |  `str`      | **Controlled Vocabulary.** Mapped mission labels (e.g., Earth Observation). |
| `un_registry` |      `str`      | Country of formal UN registration (Standardized/Cleaned). |
| `country_operator`|  `str`      | Primary country of the satellite operator/owner. |
| `is_commercial` |    `int`      | Flag: `1` if the mission includes commercial users/purposes. |
| `is_military` |      `int`      | Flag: `1` if the mission includes military/defense users. |
| `is_government` |    `int`      | Flag: `1` if the mission includes government users. |
| `is_civil` |         `int`      | Flag: `1` if the mission includes civil users. |

## **Registry Cleanup Complete**

**Summary of Operations:**
- **Normalized** 7,551 active satellite entries into a standardized, snake_case schema optimized for large-scale kinetic simulations and orbital state-vector analysis.
- **Reconstructed** 100% of missing physical and orbital data points using a **physics-informed imputation engine**, utilizing Keplerian period derivation, orbit-specific mass-ratios, and mission-based medians.
- **Calculated** operational orbital age (`sat_age_years`) and `launch_year` for every entry to support failure-risk and temporal modeling.
- **Engineered** multi-sector Boolean flags and standardized orbital regimes (LEO, MEO, GEO, Elliptical) to ensure 100% categorical and geopolitical density.
- **Enforced Integrity** via primary key deduplication and physical constraint validation (Mass > 0, Eccentricity ≥ 0), ensuring 1:1 join-readiness for the SATCAT merger.

**Next Notebook:** `satcat_cleanup.ipynb`
- Merge this physics-rich "Active Population" with the CelesTrak SATCAT to incorporate debris, rocket bodies, and radar cross-sections for a full-scale Kessler Syndrome simulation.